In [1]:
import requests
import time
import nltk
import re
import math
import numpy as np
import pandas as pd
from pandas import ExcelWriter
from nltk import sent_tokenize, word_tokenize
from nltk import pos_tag
from nltk.corpus import wordnet
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
stop_words=stopwords.words('english')
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
from nltk import pos_tag
from nltk import re
from textblob import TextBlob
from bs4 import BeautifulSoup

In [2]:
Brands=["Bosch 7 kg Fully-Automatic Front Loading Washing Machine", "Godrej 6.2 Kg Fully-Automatic Top Loading Washing Machine", "IFB 6 kg Fully-Automatic Front Loading Washing Machine", "Samsung 6.2 kg Fully-Automatic Top load Washing Machine", "Whirlpool 6.2 kg Fully-Automatic Top Loading Washing Machine", "Whirlpool 7 kg 5 Star Fully-Automatic Top Loading Washing Machine", "AmazonBasics 8 kg Semi-automatic Washing Machine", "LG 6.5 Kg 4 Star Semi-Automatic Top Loading Washing Machine", "LG 8 Kg 5 Star Semi-Automatic Top Loading Washing Machine", "Samsung 7.2 kg Semi-Automatic Top Loading Washing Machine", "Whirlpool 10.5 kg 5 Star Semi-Automatic Top Loading Washing Machine", "Whirlpool 7 Kg 5 Star Semi-Automatic Top Loading Washing Machine", "Whirlpool 7.2 Kg Semi-Automatic Top Loading Washing Machine"]

In [3]:
nReviews=[1740, 240, 990, 3879, 505, 463, 370, 715, 218, 601, 279, 1105, 417]

In [4]:
Brandlinks=["https://www.amazon.in/Bosch-Fully-Automatic-Loading-Washing-WAK24168IN/product-reviews/B00OT9CS5S/ref=cm_cr_arp_d_viewopt_srt?ie=UTF8&reviewerType=all_reviews&sortBy=recent&pageNumber={}",
"https://www.amazon.in/Godrej-Fully-Automatic-WT-EON-620/product-reviews/B07QGWHH63/ref=cm_cr_arp_d_viewopt_srt?ie=UTF8&reviewerType=all_reviews&sortBy=recent&pageNumber={}",
"https://www.amazon.in/IFB-Fully-Automatic-Diva-Aqua-SX/product-reviews/B071G3B81W/ref=cm_cr_arp_d_viewopt_srt?ie=UTF8&reviewerType=all_reviews&sortBy=recent&pageNumber={}",
"https://www.amazon.in/Samsung-Fully-Automatic-WA62M4100HY-TL-Imperial/product-reviews/B0747XV38N/ref=cm_cr_arp_d_viewopt_srt?ie=UTF8&reviewerType=all_reviews&sortBy=recent&pageNumber={}",
"https://www.amazon.in/Whirlpool-6-2-Fully-Automatic-ROYAL-2YMW/product-reviews/B07N24Z8SY/ref=cm_cr_arp_d_viewopt_srt?ie=UTF8&reviewerType=all_reviews&sortBy=recent&pageNumber={}",
"https://www.amazon.in/Whirlpool-Fully-Automatic-Loading-WHITEMAGIC-ELITE/product-reviews/B07F8S7V4Q/ref=cm_cr_arp_d_viewopt_srt?ie=UTF8&reviewerType=all_reviews&sortBy=recent&pageNumber={}",
"https://www.amazon.in/AmazonBasics-Semi-Automatic-Washing-Machine/product-reviews/B07V5W3MCB/ref=cm_cr_arp_d_viewopt_srt?ie=UTF8&reviewerType=all_reviews&sortBy=recent&pageNumber={}",
"https://www.amazon.in/LG-Semi-Automatic-Loading-P6510NBAY-Technology/product-reviews/B081J583KB/ref=cm_cr_arp_d_viewopt_srt?ie=UTF8&reviewerType=all_reviews&sortBy=recent&pageNumber={}",
"https://www.amazon.in/LG-Semi-Automatic-Loading-Washing-P8035SPMZ/product-reviews/B07WNQNQGK/ref=cm_cr_arp_d_viewopt_srt?ie=UTF8&reviewerType=all_reviews&sortBy=recent&pageNumber={}",
"https://www.amazon.in/Samsung-Semi-Automatic-Loading-Washing-WT725QPNDMP/product-reviews/B00LSJY0JE/ref=cm_cr_arp_d_viewopt_srt?ie=UTF8&reviewerType=all_reviews&sortBy=recent&pageNumber={}",
"https://www.amazon.in/Whirlpool-10-5-Semi-Automatic-ACE-XL/product-reviews/B07BNLKS7V/ref=cm_cr_arp_d_viewopt_srt?ie=UTF8&reviewerType=all_reviews&sortBy=recent&pageNumber={}",
"https://www.amazon.in/Whirlpool-SUPERB-ATOM-GREY-DAZZLE/product-reviews/B07WGD8QQT/ref=cm_cr_arp_d_viewopt_srt?ie=UTF8&reviewerType=all_reviews&sortBy=recent&pageNumber={}",
"https://www.amazon.in/Whirlpool-7-2-Ace-Supreme-Plus/product-reviews/B07P47RBC9/ref=cm_cr_arp_d_viewopt_srt?ie=UTF8&reviewerType=all_reviews&sortBy=recent&pageNumber={}"]

In [6]:
ScrapeLinks=[]
for i in range(len(Brandlinks)):
    n=math.ceil(nReviews[i]/10) #Input nReviews/10
    nPage=range(1,n+1)
    pg_links=[]
    BrandLink=Brandlinks[i]
    for p in range(len(nPage)):
        link=BrandLink.format(nPage[p])
        pg_links.append(link)
    ScrapeLinks.append(pg_links)

In [10]:
ScrapeLinks[1]

['https://www.amazon.in/Godrej-Fully-Automatic-WT-EON-620/product-reviews/B07QGWHH63/ref=cm_cr_arp_d_viewopt_srt?ie=UTF8&reviewerType=all_reviews&sortBy=recent&pageNumber=1',
 'https://www.amazon.in/Godrej-Fully-Automatic-WT-EON-620/product-reviews/B07QGWHH63/ref=cm_cr_arp_d_viewopt_srt?ie=UTF8&reviewerType=all_reviews&sortBy=recent&pageNumber=2',
 'https://www.amazon.in/Godrej-Fully-Automatic-WT-EON-620/product-reviews/B07QGWHH63/ref=cm_cr_arp_d_viewopt_srt?ie=UTF8&reviewerType=all_reviews&sortBy=recent&pageNumber=3',
 'https://www.amazon.in/Godrej-Fully-Automatic-WT-EON-620/product-reviews/B07QGWHH63/ref=cm_cr_arp_d_viewopt_srt?ie=UTF8&reviewerType=all_reviews&sortBy=recent&pageNumber=4',
 'https://www.amazon.in/Godrej-Fully-Automatic-WT-EON-620/product-reviews/B07QGWHH63/ref=cm_cr_arp_d_viewopt_srt?ie=UTF8&reviewerType=all_reviews&sortBy=recent&pageNumber=5',
 'https://www.amazon.in/Godrej-Fully-Automatic-WT-EON-620/product-reviews/B07QGWHH63/ref=cm_cr_arp_d_viewopt_srt?ie=UTF8&revi

In [90]:
#Reuqest Loop (while) and Data Scraping

cust_name=[]
review_title=[]
rating=[]
Reviews=[]
Dt=[]
Verification=[]
Helpfull=[]
Date=[]
MasterDfs=[]

for brand in range(len(ScrapeLinks)): 
    for page in range(len(ScrapeLinks[brand])):
        time.sleep(3)
        PageRequest=requests.get(ScrapeLinks[brand][page])
        while PageRequest.status_code!=200:
            PageRequest=requests.get(ScrapeLinks[brand][page])
            print(ScrapeLinks[brand][page])
            print("Requestfail")
            time.sleep(5)

        else:
            soup = BeautifulSoup(PageRequest.content, 'html5lib',from_encoding="UTF8")
            Names=soup.find_all("span",class_="a-profile-name")
            for n in range(2,len(Names)):
                cust_name.append(Names[n].text)

            ReviewTitle=soup.find_all("a",class_="review-title")
            for rt in ReviewTitle:
                review_title.append(rt.text)

            Rating=soup.find_all("i",class_="review-rating")
            for rtg in range(2,len(Rating)):
                rating.append(Rating[rtg].text)
 
            review=soup.find_all("span",class_="review-text")
            for rvw in review:
                Reviews.append(rvw.text)
            reviews=[re.sub("\\n","",k) for k in Reviews]

            date=soup.find_all("span",class_="a-size-base a-color-secondary review-date")
            for dt in range(2,len(date)):
                Date.append(date[dt].string)

            vf=soup.find_all("span",class_="a-size-mini a-color-state a-text-bold")
            for vr in range(len(vf)):
                Verification.append(vf[vr].text)

            hf=soup.find_all("span",class_="a-size-base a-color-tertiary cr-vote-text")
            for hp in range(len(hf)):
                Helpfull.append(hf[hp].text)

            print(ScrapeLinks[brand][page])
            print(PageRequest)

            print(len(Date))
            print(len(cust_name))
            print(len(rating))
            print(len(review_title))
            print(len(reviews))

    review_title=[re.sub("\\n","",rt) for rt in review_title]
    rating=[rating[rtg][0:3] for rtg in range(len(rating))]
    Date=[re.sub("Reviewed in India on ","",d) for d in Date]
    Helpfull=[re.sub("people found this helpful","",h) for h in Helpfull]
    
    BrandDf=pd.DataFrame()
    BrandDf["Date"]=Date
    BrandDf["Name"]=cust_name
    BrandDf["Ratings"]=rating
    BrandDf["Brand"]=Brands[brand]
    BrandDf["Title"]=review_title
    BrandDf["Reviews"]=reviews
    
    MasterDfs.append(BrandDf)
    
    cust_name=[]
    review_title=[]
    rating=[]
    Reviews=[]
    reviews=[]
    Date=[]
    Verification=[]
    Helpfull=[] 

https://www.amazon.in/Bosch-Fully-Automatic-Loading-Washing-WAK24168IN/product-reviews/B00OT9CS5S/ref=cm_cr_arp_d_viewopt_srt?ie=UTF8&reviewerType=all_reviews&sortBy=recent&pageNumber=1
<Response [200]>
10
10
10
10
10
https://www.amazon.in/Bosch-Fully-Automatic-Loading-Washing-WAK24168IN/product-reviews/B00OT9CS5S/ref=cm_cr_arp_d_viewopt_srt?ie=UTF8&reviewerType=all_reviews&sortBy=recent&pageNumber=2
<Response [200]>
20
20
20
20
20
https://www.amazon.in/Bosch-Fully-Automatic-Loading-Washing-WAK24168IN/product-reviews/B00OT9CS5S/ref=cm_cr_arp_d_viewopt_srt?ie=UTF8&reviewerType=all_reviews&sortBy=recent&pageNumber=3
<Response [200]>
30
30
30
30
30
https://www.amazon.in/Bosch-Fully-Automatic-Loading-Washing-WAK24168IN/product-reviews/B00OT9CS5S/ref=cm_cr_arp_d_viewopt_srt?ie=UTF8&reviewerType=all_reviews&sortBy=recent&pageNumber=4
<Response [200]>
40
40
40
40
40
https://www.amazon.in/Bosch-Fully-Automatic-Loading-Washing-WAK24168IN/product-reviews/B00OT9CS5S/ref=cm_cr_arp_d_viewopt_srt?ie

https://www.amazon.in/Bosch-Fully-Automatic-Loading-Washing-WAK24168IN/product-reviews/B00OT9CS5S/ref=cm_cr_arp_d_viewopt_srt?ie=UTF8&reviewerType=all_reviews&sortBy=recent&pageNumber=21
<Response [200]>
210
210
210
210
210
https://www.amazon.in/Bosch-Fully-Automatic-Loading-Washing-WAK24168IN/product-reviews/B00OT9CS5S/ref=cm_cr_arp_d_viewopt_srt?ie=UTF8&reviewerType=all_reviews&sortBy=recent&pageNumber=22
<Response [200]>
220
220
220
220
220
https://www.amazon.in/Bosch-Fully-Automatic-Loading-Washing-WAK24168IN/product-reviews/B00OT9CS5S/ref=cm_cr_arp_d_viewopt_srt?ie=UTF8&reviewerType=all_reviews&sortBy=recent&pageNumber=23
Requestfail
https://www.amazon.in/Bosch-Fully-Automatic-Loading-Washing-WAK24168IN/product-reviews/B00OT9CS5S/ref=cm_cr_arp_d_viewopt_srt?ie=UTF8&reviewerType=all_reviews&sortBy=recent&pageNumber=23
Requestfail
https://www.amazon.in/Bosch-Fully-Automatic-Loading-Washing-WAK24168IN/product-reviews/B00OT9CS5S/ref=cm_cr_arp_d_viewopt_srt?ie=UTF8&reviewerType=all_rev

https://www.amazon.in/Bosch-Fully-Automatic-Loading-Washing-WAK24168IN/product-reviews/B00OT9CS5S/ref=cm_cr_arp_d_viewopt_srt?ie=UTF8&reviewerType=all_reviews&sortBy=recent&pageNumber=39
Requestfail
https://www.amazon.in/Bosch-Fully-Automatic-Loading-Washing-WAK24168IN/product-reviews/B00OT9CS5S/ref=cm_cr_arp_d_viewopt_srt?ie=UTF8&reviewerType=all_reviews&sortBy=recent&pageNumber=39
<Response [200]>
390
390
390
390
390
https://www.amazon.in/Bosch-Fully-Automatic-Loading-Washing-WAK24168IN/product-reviews/B00OT9CS5S/ref=cm_cr_arp_d_viewopt_srt?ie=UTF8&reviewerType=all_reviews&sortBy=recent&pageNumber=40
Requestfail
https://www.amazon.in/Bosch-Fully-Automatic-Loading-Washing-WAK24168IN/product-reviews/B00OT9CS5S/ref=cm_cr_arp_d_viewopt_srt?ie=UTF8&reviewerType=all_reviews&sortBy=recent&pageNumber=40
<Response [200]>
400
400
400
400
400
https://www.amazon.in/Bosch-Fully-Automatic-Loading-Washing-WAK24168IN/product-reviews/B00OT9CS5S/ref=cm_cr_arp_d_viewopt_srt?ie=UTF8&reviewerType=all_rev

https://www.amazon.in/Bosch-Fully-Automatic-Loading-Washing-WAK24168IN/product-reviews/B00OT9CS5S/ref=cm_cr_arp_d_viewopt_srt?ie=UTF8&reviewerType=all_reviews&sortBy=recent&pageNumber=62
<Response [200]>
620
620
620
620
620
https://www.amazon.in/Bosch-Fully-Automatic-Loading-Washing-WAK24168IN/product-reviews/B00OT9CS5S/ref=cm_cr_arp_d_viewopt_srt?ie=UTF8&reviewerType=all_reviews&sortBy=recent&pageNumber=63
Requestfail
https://www.amazon.in/Bosch-Fully-Automatic-Loading-Washing-WAK24168IN/product-reviews/B00OT9CS5S/ref=cm_cr_arp_d_viewopt_srt?ie=UTF8&reviewerType=all_reviews&sortBy=recent&pageNumber=63
<Response [200]>
630
630
630
630
630
https://www.amazon.in/Bosch-Fully-Automatic-Loading-Washing-WAK24168IN/product-reviews/B00OT9CS5S/ref=cm_cr_arp_d_viewopt_srt?ie=UTF8&reviewerType=all_reviews&sortBy=recent&pageNumber=64
<Response [200]>
640
640
640
640
640
https://www.amazon.in/Bosch-Fully-Automatic-Loading-Washing-WAK24168IN/product-reviews/B00OT9CS5S/ref=cm_cr_arp_d_viewopt_srt?ie=

https://www.amazon.in/Bosch-Fully-Automatic-Loading-Washing-WAK24168IN/product-reviews/B00OT9CS5S/ref=cm_cr_arp_d_viewopt_srt?ie=UTF8&reviewerType=all_reviews&sortBy=recent&pageNumber=86
Requestfail
https://www.amazon.in/Bosch-Fully-Automatic-Loading-Washing-WAK24168IN/product-reviews/B00OT9CS5S/ref=cm_cr_arp_d_viewopt_srt?ie=UTF8&reviewerType=all_reviews&sortBy=recent&pageNumber=86
<Response [200]>
860
860
860
860
860
https://www.amazon.in/Bosch-Fully-Automatic-Loading-Washing-WAK24168IN/product-reviews/B00OT9CS5S/ref=cm_cr_arp_d_viewopt_srt?ie=UTF8&reviewerType=all_reviews&sortBy=recent&pageNumber=87
Requestfail
https://www.amazon.in/Bosch-Fully-Automatic-Loading-Washing-WAK24168IN/product-reviews/B00OT9CS5S/ref=cm_cr_arp_d_viewopt_srt?ie=UTF8&reviewerType=all_reviews&sortBy=recent&pageNumber=87
<Response [200]>
870
870
870
870
870
https://www.amazon.in/Bosch-Fully-Automatic-Loading-Washing-WAK24168IN/product-reviews/B00OT9CS5S/ref=cm_cr_arp_d_viewopt_srt?ie=UTF8&reviewerType=all_rev

...

https://www.amazon.in/Whirlpool-SUPERB-ATOM-GREY-DAZZLE/product-reviews/B07WGD8QQT/ref=cm_cr_arp_d_viewopt_srt?ie=UTF8&reviewerType=all_reviews&sortBy=recent&pageNumber=49
Requestfail
https://www.amazon.in/Whirlpool-SUPERB-ATOM-GREY-DAZZLE/product-reviews/B07WGD8QQT/ref=cm_cr_arp_d_viewopt_srt?ie=UTF8&reviewerType=all_reviews&sortBy=recent&pageNumber=49
Requestfail
https://www.amazon.in/Whirlpool-SUPERB-ATOM-GREY-DAZZLE/product-reviews/B07WGD8QQT/ref=cm_cr_arp_d_viewopt_srt?ie=UTF8&reviewerType=all_reviews&sortBy=recent&pageNumber=49
<Response [200]>
490
490
490
490
490
https://www.amazon.in/Whirlpool-SUPERB-ATOM-GREY-DAZZLE/product-reviews/B07WGD8QQT/ref=cm_cr_arp_d_viewopt_srt?ie=UTF8&reviewerType=all_reviews&sortBy=recent&pageNumber=50
Requestfail
https://www.amazon.in/Whirlpool-SUPERB-ATOM-GREY-DAZZLE/product-reviews/B07WGD8QQT/ref=cm_cr_arp_d_viewopt_srt?ie=UTF8&reviewerType=all_reviews&sortBy=recent&pageNumber=50
<Response [200]>
500
500
500
500
500
https://www.amazon.in/Whirlpoo

https://www.amazon.in/Whirlpool-SUPERB-ATOM-GREY-DAZZLE/product-reviews/B07WGD8QQT/ref=cm_cr_arp_d_viewopt_srt?ie=UTF8&reviewerType=all_reviews&sortBy=recent&pageNumber=66
<Response [200]>
660
660
660
660
660
https://www.amazon.in/Whirlpool-SUPERB-ATOM-GREY-DAZZLE/product-reviews/B07WGD8QQT/ref=cm_cr_arp_d_viewopt_srt?ie=UTF8&reviewerType=all_reviews&sortBy=recent&pageNumber=67
Requestfail
https://www.amazon.in/Whirlpool-SUPERB-ATOM-GREY-DAZZLE/product-reviews/B07WGD8QQT/ref=cm_cr_arp_d_viewopt_srt?ie=UTF8&reviewerType=all_reviews&sortBy=recent&pageNumber=67
Requestfail
https://www.amazon.in/Whirlpool-SUPERB-ATOM-GREY-DAZZLE/product-reviews/B07WGD8QQT/ref=cm_cr_arp_d_viewopt_srt?ie=UTF8&reviewerType=all_reviews&sortBy=recent&pageNumber=67
Requestfail
https://www.amazon.in/Whirlpool-SUPERB-ATOM-GREY-DAZZLE/product-reviews/B07WGD8QQT/ref=cm_cr_arp_d_viewopt_srt?ie=UTF8&reviewerType=all_reviews&sortBy=recent&pageNumber=67
<Response [200]>
670
670
670
670
670
https://www.amazon.in/Whirlpoo

https://www.amazon.in/Whirlpool-SUPERB-ATOM-GREY-DAZZLE/product-reviews/B07WGD8QQT/ref=cm_cr_arp_d_viewopt_srt?ie=UTF8&reviewerType=all_reviews&sortBy=recent&pageNumber=85
<Response [200]>
850
850
850
850
850
https://www.amazon.in/Whirlpool-SUPERB-ATOM-GREY-DAZZLE/product-reviews/B07WGD8QQT/ref=cm_cr_arp_d_viewopt_srt?ie=UTF8&reviewerType=all_reviews&sortBy=recent&pageNumber=86
Requestfail
https://www.amazon.in/Whirlpool-SUPERB-ATOM-GREY-DAZZLE/product-reviews/B07WGD8QQT/ref=cm_cr_arp_d_viewopt_srt?ie=UTF8&reviewerType=all_reviews&sortBy=recent&pageNumber=86
<Response [200]>
860
860
860
860
860
https://www.amazon.in/Whirlpool-SUPERB-ATOM-GREY-DAZZLE/product-reviews/B07WGD8QQT/ref=cm_cr_arp_d_viewopt_srt?ie=UTF8&reviewerType=all_reviews&sortBy=recent&pageNumber=87
<Response [200]>
870
870
870
870
870
https://www.amazon.in/Whirlpool-SUPERB-ATOM-GREY-DAZZLE/product-reviews/B07WGD8QQT/ref=cm_cr_arp_d_viewopt_srt?ie=UTF8&reviewerType=all_reviews&sortBy=recent&pageNumber=88
Requestfail
https

https://www.amazon.in/Whirlpool-SUPERB-ATOM-GREY-DAZZLE/product-reviews/B07WGD8QQT/ref=cm_cr_arp_d_viewopt_srt?ie=UTF8&reviewerType=all_reviews&sortBy=recent&pageNumber=111
<Response [200]>
1105
1105
1105
1105
1105
https://www.amazon.in/Whirlpool-7-2-Ace-Supreme-Plus/product-reviews/B07P47RBC9/ref=cm_cr_arp_d_viewopt_srt?ie=UTF8&reviewerType=all_reviews&sortBy=recent&pageNumber=1
Requestfail
https://www.amazon.in/Whirlpool-7-2-Ace-Supreme-Plus/product-reviews/B07P47RBC9/ref=cm_cr_arp_d_viewopt_srt?ie=UTF8&reviewerType=all_reviews&sortBy=recent&pageNumber=1
Requestfail
https://www.amazon.in/Whirlpool-7-2-Ace-Supreme-Plus/product-reviews/B07P47RBC9/ref=cm_cr_arp_d_viewopt_srt?ie=UTF8&reviewerType=all_reviews&sortBy=recent&pageNumber=1
<Response [200]>
10
10
10
10
10
https://www.amazon.in/Whirlpool-7-2-Ace-Supreme-Plus/product-reviews/B07P47RBC9/ref=cm_cr_arp_d_viewopt_srt?ie=UTF8&reviewerType=all_reviews&sortBy=recent&pageNumber=2
<Response [200]>
20
20
20
20
20
https://www.amazon.in/Whi

https://www.amazon.in/Whirlpool-7-2-Ace-Supreme-Plus/product-reviews/B07P47RBC9/ref=cm_cr_arp_d_viewopt_srt?ie=UTF8&reviewerType=all_reviews&sortBy=recent&pageNumber=26
Requestfail
https://www.amazon.in/Whirlpool-7-2-Ace-Supreme-Plus/product-reviews/B07P47RBC9/ref=cm_cr_arp_d_viewopt_srt?ie=UTF8&reviewerType=all_reviews&sortBy=recent&pageNumber=26
<Response [200]>
260
260
260
260
260
https://www.amazon.in/Whirlpool-7-2-Ace-Supreme-Plus/product-reviews/B07P47RBC9/ref=cm_cr_arp_d_viewopt_srt?ie=UTF8&reviewerType=all_reviews&sortBy=recent&pageNumber=27
<Response [200]>
270
270
270
270
270
https://www.amazon.in/Whirlpool-7-2-Ace-Supreme-Plus/product-reviews/B07P47RBC9/ref=cm_cr_arp_d_viewopt_srt?ie=UTF8&reviewerType=all_reviews&sortBy=recent&pageNumber=28
<Response [200]>
280
280
280
280
280
https://www.amazon.in/Whirlpool-7-2-Ace-Supreme-Plus/product-reviews/B07P47RBC9/ref=cm_cr_arp_d_viewopt_srt?ie=UTF8&reviewerType=all_reviews&sortBy=recent&pageNumber=29
Requestfail
https://www.amazon.i

In [93]:
MasterDfs[1]

,Date,Name,Ratings,Brand,Title,Reviews
0,14 July 2020,D K Maurya,3.0,Godrej 6.2 Kg Fully-Automatic Top Loading Wash...,Nice,Little more than flipkart price
1,11 July 2020,Poornima,1.0,Godrej 6.2 Kg Fully-Automatic Top Loading Wash...,POOR PRODUCT .NOT UPTO THE BRAND NAME,We purchased the product 2 years back...
2,10 July 2020,Shirin Mirza,1.0,Godrej 6.2 Kg Fully-Automatic Top Loading Wash...,Child lok dose not work,Dislike
3,9 July 2020,Darsh,5.0,Godrej 6.2 Kg Fully-Automatic Top Loading Wash...,durable,amazing product.i would like to give 1...
4,5 July 2020,Kapil bhadwaj,5.0,Godrej 6.2 Kg Fully-Automatic Top Loading Wash...,MADE IN INDIA,It's a indian brand I like it. Be Indi...
...,...,...,...,...,...,...
235,15 June 2019,Amazon Customer,1.0,Godrej 6.2 Kg Fully-Automatic Top Loading Wash...,No seperate wash option,not giving a seperate wash only optio...
236,7 June 2019,M bhanu chand,4.0,Godrej 6.2 Kg Fully-Automatic Top Loading Wash...,Nice worthy,Good
237,3 June 2019,Rohit,5.0,Godrej 6.2 Kg Fully-Automatic Top Loading Wash...,Good Product,"Good machine, value for money.Does not..."
238,30 May 2019,rupa rani,1.0,Godrej 6.2 Kg Fully-Automatic Top Loading Wash...,Technicains are too bad,Product is good but yoir technicians a...


In [94]:
finalDf=pd.concat(MasterDfs)

In [95]:
finalDf

,Date,Name,Ratings,Brand,Title,Reviews
0,18 July 2020,Gali Gopinadh,1.0,Bosch 7 kg Fully-Automatic Front Loading Washi...,Some Options Not Working Properly,It's tuf to use. Some options are not ...
1,18 July 2020,Naman,2.0,Bosch 7 kg Fully-Automatic Front Loading Washi...,Internal technical issues in machine.,"Faulty product received, machine not w..."
2,17 July 2020,Sai Sravanthi,5.0,Bosch 7 kg Fully-Automatic Front Loading Washi...,Really worth every penny. Go for it!!!!,I have received the product on 15th ju...
3,17 July 2020,Nagendra Kumar,1.0,Bosch 7 kg Fully-Automatic Front Loading Washi...,Cloudtail and Amazon cheated me with t...,I've bought model WAK24168IN. On Amazo...
4,17 July 2020,santosh,5.0,Bosch 7 kg Fully-Automatic Front Loading Washi...,nice,great onhe
...,...,...,...,...,...,...
412,7 April 2019,Amazon Customer,5.0,Whirlpool 7.2 Kg Semi-Automatic Top Loading Wa...,Product is good and easy to use with n...,Good prdt
413,7 April 2019,Nikks,4.0,Whirlpool 7.2 Kg Semi-Automatic Top Loading Wa...,Meets exceptation,Works as excepted. No complaint.
414,2 February 2019,Amazon Customer,5.0,Whirlpool 7.2 Kg Semi-Automatic Top Loading Wa...,GOOD,good service. satisfied with product
415,28 January 2019,bhupinder singh,4.0,Whirlpool 7.2 Kg Semi-Automatic Top Loading Wa...,Amazing,Good product


In [96]:
finalDf.to_excel('WashingMachineCorpus.xlsx')

In [54]:
FinalDf

,Date,Name,Ratings,Title,Reviews
0,13 July 2020,Shailesh,4.0,Worth buying!,Product as per specifications
1,13 July 2020,Amazon Customer,5.0,Voltas ac,Good product.
2,10 July 2020,R P Khandelwal,4.0,Nothing Most Important,It's good and working best as known pr...
3,8 July 2020,Rakesh,1.0,"Worst After Sales service , Voltas loo...","I bought 2 AC , both from same vendor ..."
4,7 July 2020,Nirav Gohel,4.0,Best In Price...,Best AC Value For Money...
...,...,...,...,...,...
575,9 June 2015,ARIJIT ROY,5.0,"Good product, Bright colour","Good product,Bright colour,used almost..."
576,6 June 2015,Rudra prasad Malla,5.0,good ac but have installation charges,Very good ac but installation changes ...
577,6 May 2015,Sagar,1.0,DO NOT BUY ANY VOLTAS AC - POOR AFTER ...,DO NOT BUY ANY VOLTAS AC - POOR AFTER ...
578,11 April 2015,Ramesh Mhadgut,4.0,Four Stars,Very very good


In [55]:
FinalDf.to_excel("Voltas1Ton3StarSplitACAmzReviews.xlsx")

In [176]:
#Single

cust_name=[]
review_title=[]
rating=[]
Reviews=[]
Dt=[]
Verification=[]
Helpfull=[]
Date=[]

Whirlpool=requests.get(pg_links[3])
soup = BeautifulSoup(Whirlpool.content, 'html.parser',from_encoding="UTF8")

Names=soup.find_all("span",class_="a-profile-name")
for n in range(2,len(Names)):
    cust_name.append(Names[n].text)

ReviewTitle=soup.find_all("a",class_="review-title")
for rt in ReviewTitle:
    review_title.append(rt.text)

Rating=soup.find_all("i",class_="review-rating")
for rtg in range(2,len(Rating)):
    rating.append(Rating[rtg].text)

review=soup.find_all("span",class_="review-text")
for rvw in review:
    Reviews.append(rvw.text)
reviews=[re.sub("\\n","",k) for k in Reviews]

date=soup.find_all("span",class_="a-size-base a-color-secondary review-date")
for dt in range(2,len(date)):
    Date.append(date[dt].string)

vf=soup.find_all("span",class_="a-size-mini a-color-state a-text-bold")
for vr in range(len(vf)):
    Verification.append(vf[vr].text)

hf=soup.find_all("span",class_="a-size-base a-color-tertiary cr-vote-text")
for hp in range(len(hf)):
    Helpfull.append(hf[hp].text)

# review_title=[re.sub("\\n","",rt) for rt in review_title]
# rating=[rating[rtg][0:3] for rtg in range(len(rating))]
# Date=[re.sub("Reviewed in India on ","",d) for d in Dt]
# Helpfull=[re.sub("people found this helpful","",h) for h in Helpfull]